In [ ]:
!pip install diffusers --upgrade
!pip install invisible_watermark transformers accelerate safetensors

In [ ]:
#@markdown ####**Settings:**



from diffusers import DiffusionPipeline
from diffusers import (
    DDIMScheduler,
    DPMSolverMultistepScheduler,
)
import torch
import time

# seed = torch.Generator("cuda").manual_seed(1517224853)

# SDXL
model_path = "stabilityai/stable-diffusion-xl-base-1.0" #@param{type: 'string'}
refiner_path = "stabilityai/stable-diffusion-xl-refiner-1.0" #@param{type: 'string'}

ddim = DDIMScheduler.from_pretrained(model_path, subfolder="scheduler")
DPM_2M_Karras = DPMSolverMultistepScheduler.from_pretrained(model_path, subfolder="scheduler", use_karras_sigmas=True)
DPM_2M_SDE_Karras = DPMSolverMultistepScheduler.from_pretrained(model_path, subfolder="scheduler", use_karras_sigmas=True, algorithm_type="sde-dpmsolver++")


scheduler = DPM_2M_Karras #@param ["ddim", "DPM_2M_Karras", "DPM_2M_SDE_Karras"] {type:"raw"}

# load both base & refiner
base = DiffusionPipeline.from_pretrained(
    model_path, scheduler=scheduler, torch_dtype=torch.float16, variant="fp16", use_safetensors=True
)
base.to("cuda")
refiner = DiffusionPipeline.from_pretrained(
    refiner_path,
    scheduler=scheduler,
    text_encoder_2=base.text_encoder_2,
    vae=base.vae,
    torch_dtype=torch.float16,
    use_safetensors=True,
    variant="fp16",
)
refiner.to("cuda")

# Define how many steps and what % of steps to be run on each experts (80/20) here
n_steps = 20 #@param{type: 'number'}
width = 1024 #@param{type: 'number'}
height = 1024 #@param{type: 'number'}
high_noise_frac = 0.8

prompt = "" #@param{type: 'string'}
negative_prompt = "" #@param{type: 'string'}

# run both experts
image = base(
    prompt=prompt,
    width=width,
    height=height,
    negative_prompt=negative_prompt,
    num_inference_steps=n_steps,
    denoising_end=high_noise_frac,
    output_type="latent",
).images
image = refiner(
    prompt=prompt,
    negative_prompt=negative_prompt,
    num_inference_steps=n_steps,
    denoising_start=high_noise_frac,
    image=image,
).images[0]

image.save("random_"+ str(int(time.time())) +".png")
